In [1]:
import pandas as pd
from rdkit import Chem
import seaborn as sns
from tqdm import tqdm
import numpy as np

In [2]:
tqdm.pandas()


In [3]:
chembl = pd.read_csv("ChEMBL/ChEMBL3201.tsv", sep="\t")

In [4]:
chembl

,ChEMBL ID,Name,Synonyms,Type,Max Phase,Molecular Weight,Targets,Bioactivities,AlogP,Polar Surface Area,...,Np Likeness Score,Molecular Species,Molecular Formula,Smiles,Inchi Key,Inchi,Withdrawn Flag,Orphan,Records Key,Records Name
0,CHEMBL3989951,NAVOXIMOD,GDC-0919|IDO-IN-7|NAVOXIMOD|NLG-919|NLG919|RG-...,Small molecule,2.0,316.38,17,126,2.89,58.28,...,0.05,NEUTRAL,C18H21FN2O2,O[C@H](C[C@H]1c2c(F)cccc2-c2cncn21)[C@H]1CC[C@...,YGACXVRLDHEXKY-WXRXAMBDSA-N,InChI=1S/C18H21FN2O2/c19-14-3-1-2-13-16-9-20-1...,False,0,"['NAVOXIMOD', 'NLG-919', '15; NLG919', 'GDC-09...","['NAVOXIMOD', '(1R,4r)-4-((R)-2-((S)-6-fluoro-..."
1,CHEMBL46917,CARBARIL,CARBARIL|CARBARILO|CARBARYL|CARYLDERM|CLINICID...,Small molecule,4.0,201.22,74,163,2.56,38.33,...,-0.32,NEUTRAL,C12H11NO2,CNC(=O)Oc1cccc2ccccc12,CVXBEEMKQHEXEN-UHFFFAOYSA-N,InChI=1S/C12H11NO2/c1-13-12(14)15-11-8-4-6-9-5...,True,0,"['Carboryl', 'carbaryl', '86', 'Carbaryl 1', '...","['Carboryl', 'Carbaryl', 'naphthalen-1-yl meth..."
2,CHEMBL359744,DOXORUBICIN HYDROCHLORIDE,ADRIABLASTINA CS|ADRIAMYCIN|ADRIAMYCIN PFS|ADR...,Small molecule,4.0,579.99,260,1586,0.00,206.07,...,1.81,BASE,C27H30ClNO11,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,MWWSFMDVAYGXBV-RUELKSSGSA-N,InChI=1S/C27H29NO11.ClH/c1-10-22(31)13(28)6-17...,False,0,"['1', 'Adriamycin', 'Dox', 'Doxorubicin HCl', ...",['10-(4-Amino-5-hydroxy-6-methyl-tetrahydro-py...
3,CHEMBL63857,PHENOLPHTHALEIN,ALOPHEN|FENOLFTALEINA|PHENOLPHTALEINE|PHENOLPH...,Small molecule,4.0,318.33,65,129,3.56,66.76,...,0.41,NEUTRAL,C20H14O4,O=C1OC(c2ccc(O)cc2)(c2ccc(O)cc2)c2ccccc21,KJFMBFZCATUALV-UHFFFAOYSA-N,InChI=1S/C20H14O4/c21-15-9-5-13(6-10-15)20(14-...,True,0,"['SID26747046', '1a', '25', '5', 'SID17389766'...","['SID26747046', '3,3-Bis-(4-hydroxy-phenyl)-3H..."
4,CHEMBL70971,CARBENDAZIM,A 118|BAS-67054F|BAVISTIN|CARBENDAZIM|DERROPRE...,Small molecule,1.0,191.19,166,770,1.74,67.01,...,-1.10,NEUTRAL,C9H9N3O2,COC(=O)Nc1nc2ccccc2[nH]1,TWFZGCMQGLPBSX-UHFFFAOYSA-N,InChI=1S/C9H9N3O2/c1-14-9(13)12-8-10-6-4-2-3-5...,False,0,"['3', 'carbendazim', 'Carbendazim', 'C', 'Carb...",['(1H-Benzoimidazol-2-yl)-carbamic acid methyl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,CHEMBL963,OXYMORPHONE,"14-HYDROXYDIHYDROMORPHINONE|7,8-DIHYDRO-14-HYD...",Small molecule,4.0,301.34,40,103,0.75,70.00,...,2.20,NEUTRAL,C17H19NO4,CN1CC[C@]23c4c5ccc(O)c4O[C@H]2C(=O)CC[C@@]3(O)...,UQCNKQCJZOAFTQ-ISWURRPUSA-N,InChI=1S/C17H19NO4/c1-18-7-6-16-13-9-2-3-10(19...,False,0,"['Oxymorphone', 'oxymor', 'oxymorphone', '1a',...","['10,17-dihydroxy-4-methyl-(13R,17S)-12-oxa-4-..."
725,CHEMBL498416,L-KYNURENINE,L-Kynurenine,Small molecule,1.0,208.22,9,28,0.25,106.41,...,0.67,ZWITTERION,C10H12N2O3,Nc1ccccc1C(=O)C[C@H](N)C(=O)O,YGPSJZOEDVAXAB-QMMMGPOBSA-N,InChI=1S/C10H12N2O3/c11-7-4-2-1-3-6(7)9(13)5-8...,False,0,"['17', 'L-Kyn', 'SID26756700', 'Glu/Asp family...","['L-2-Amino-3-(2-aminobenzoyl)propionic acid',..."
726,CHEMBL1773683,EZUTROMID,BMN 195|BMN-195|BMN195|EZUTROMID|SMT C1100|SMT...,Small molecule,2.0,337.40,28,110,4.44,60.17,...,-1.57,NEUTRAL,C19H15NO3S,CCS(=O)(=O)c1ccc2oc(-c3ccc4ccccc4c3)nc2c1,KSGCNXAZROJSNW-UHFFFAOYSA-N,"InChI=1S/C19H15NO3S/c1-2-24(21,22)16-9-10-18-1...",False,0,"['1', 'EZUTROMID', '1', '3', 'EZUTROMID', '1; ...",['5-(Ethylsulfonyl)-2-(naphthalen-2-yl)benzo[d...
727,CHEMBL656,OXYCODONE,(-)-14-HYDROXYDIHYDROCODEINONE|14-HYDROXYDIHYD...,Small molecule,4.0,315.37,86,215,1.05,59.00,...,1.91,NEUTRAL,C18H21NO4,COc1ccc2c3c1O[C@H]1C(=O)CC[C@@]4(O)[C@@H](C2)N...,BRUQQQPBMZOVGD-XFKAJCMBSA-N,InChI=1S/C18H21NO4/c1-19-8-7-17-14-10-3-4-12(2...,True,0,"['Oxycodone', '256', '25', '14-hydroxydi-hydro...","['17-hydroxy-10-methoxy-4-methyl-(1S,5R,13R,17..."


In [5]:
chembl.columns

Index(['ChEMBL ID', 'Name', 'Synonyms', 'Type', 'Max Phase',
       'Molecular Weight', 'Targets', 'Bioactivities', 'AlogP',
       'Polar Surface Area', 'HBA', 'HBD', '#RO5 Violations',
       '#Rotatable Bonds', 'Passes Ro3', 'QED Weighted', 'CX Acidic pKa',
       'CX Basic pKa', 'CX LogP', 'CX LogD', 'Aromatic Rings',
       'Structure Type', 'Inorganic Flag', 'Heavy Atoms', 'HBA (Lipinski)',
       'HBD (Lipinski)', '#RO5 Violations (Lipinski)',
       'Molecular Weight (Monoisotopic)', 'Np Likeness Score',
       'Molecular Species', 'Molecular Formula', 'Smiles', 'Inchi Key',
       'Inchi', 'Withdrawn Flag', 'Orphan', 'Records Key', 'Records Name'],
      dtype='object')

In [6]:
n_mol = chembl["ChEMBL ID"].nunique()
n_mol

729

In [7]:

# from Pat Walters method
def standardize_smiles(smiles):
    try:
        # follows the steps in
        # https://github.com/greglandrum/RSC_OpenScience_Standardization_202104/blob/main/MolStandardize%20pieces.ipynb
        # as described **excellently** (by Greg) in
        # https://www.youtube.com/watch?v=eWTApNX8dJQ
        mol = Chem.MolFromSmiles(smiles)
    
        # removeHs, disconnect metal atoms, normalize the molecule, reionize the molecule
        clean_mol = rdMolStandardize.Cleanup(mol)
    
        # if many fragments, get the "parent" (the actual mol we are interested in) 
        parent_clean_mol = rdMolStandardize.FragmentParent(clean_mol)
    
        # try to neutralize molecule
        uncharger = rdMolStandardize.Uncharger()  # annoying, but necessary as no convenience method exists
        uncharged_parent_clean_mol = uncharger.uncharge(parent_clean_mol)
    
        # note that no attempt is made at reionization at this step
        # nor at ionization at some pH (rdkit has no pKa caculator)
        # the main aim to represent all molecules from different sources
        # in a (single) standard way, for use in ML, catalogue, etc.
    
        te = rdMolStandardize.TautomerEnumerator()  # idem
        taut_uncharged_parent_clean_mol = te.Canonicalize(uncharged_parent_clean_mol)
    
        return Chem.MolToSmiles(taut_uncharged_parent_clean_mol)
    except:
        return pd.NA

In [8]:
# standardisation of SMILES
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.rdBase import BlockLogs
import mols2grid

In [9]:

with BlockLogs():
    chembl["CANONICAL_SMILES"] = chembl["Smiles"].progress_apply(lambda x: standardize_smiles(x))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 729/729 [00:06<00:00, 110.48it/s]


In [10]:
def smi_to_inchikey(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return Chem.MolToInchiKey(mol)
    except:
        return pd.NA

In [11]:
# compute inchikeys
with BlockLogs():
    chembl["INCHIKEY"] = chembl["CANONICAL_SMILES"].progress_apply(lambda x: smi_to_inchikey(x))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 729/729 [00:00<00:00, 4349.58it/s]


In [12]:
chembl.INCHIKEY.nunique()

722

In [13]:
chembl.dropna(subset="INCHIKEY", inplace=True)

In [14]:
# now clean up 
keep_cols = ["CANONICAL_SMILES", "INCHIKEY", "Name"]

In [15]:
clean_active = chembl[keep_cols]
clean_active.rename(columns={ "Name": "common_name"}, inplace=True)



/tmp/ipykernel_24516/3609579136.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_active.rename(columns={ "Name": "common_name"}, inplace=True)


In [16]:
clean_active_sorted = clean_active.sort_values(["common_name"], ascending=[False]) # keep the ones with names if possible
clean_deduped = clean_active_sorted.drop_duplicates(subset="INCHIKEY", keep="first")

In [17]:
clean_active.head()

,CANONICAL_SMILES,INCHIKEY,common_name
0,O[C@H](C[C@H]1c2c(F)cccc2-c2cncn21)[C@H]1CC[C@...,YGACXVRLDHEXKY-WXRXAMBDSA-N,NAVOXIMOD
1,CNC(=O)Oc1cccc2ccccc12,CVXBEEMKQHEXEN-UHFFFAOYSA-N,CARBARIL
2,COc1cccc2c(O)c3c(c(O)c12)C(=O)C1=C(C[C@@](O)(C...,XZTGINNGSQCYPR-TZSSRYMLSA-N,DOXORUBICIN HYDROCHLORIDE
3,O=C1OC(c2ccc(O)cc2)(c2ccc(O)cc2)c2ccccc21,KJFMBFZCATUALV-UHFFFAOYSA-N,PHENOLPHTHALEIN
4,COC(=O)Nc1nc2ccccc2[nH]1,TWFZGCMQGLPBSX-UHFFFAOYSA-N,CARBENDAZIM


In [18]:
clean_deduped["dataset"] = "ChEMBL_curated"
clean_deduped["active"] = True
clean_deduped["action_type"] = "agonist"

/tmp/ipykernel_24516/2591073474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_deduped["dataset"] = "ChEMBL_curated"
/tmp/ipykernel_24516/2591073474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_deduped["active"] = True
/tmp/ipykernel_24516/2591073474.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [19]:
clean_deduped.to_csv("processed/chembl_active.csv", index=False)

In [20]:
clean_active

,CANONICAL_SMILES,INCHIKEY,common_name
0,O[C@H](C[C@H]1c2c(F)cccc2-c2cncn21)[C@H]1CC[C@...,YGACXVRLDHEXKY-WXRXAMBDSA-N,NAVOXIMOD
1,CNC(=O)Oc1cccc2ccccc12,CVXBEEMKQHEXEN-UHFFFAOYSA-N,CARBARIL
2,COc1cccc2c(O)c3c(c(O)c12)C(=O)C1=C(C[C@@](O)(C...,XZTGINNGSQCYPR-TZSSRYMLSA-N,DOXORUBICIN HYDROCHLORIDE
3,O=C1OC(c2ccc(O)cc2)(c2ccc(O)cc2)c2ccccc21,KJFMBFZCATUALV-UHFFFAOYSA-N,PHENOLPHTHALEIN
4,COC(=O)Nc1nc2ccccc2[nH]1,TWFZGCMQGLPBSX-UHFFFAOYSA-N,CARBENDAZIM
...,...,...,...
724,CN1CC[C@]23c4c5ccc(O)c4OC2C(=O)CC[C@@]3(O)[C@H...,UQCNKQCJZOAFTQ-VSYOVCOZSA-N,OXYMORPHONE
725,Nc1ccccc1C(=O)CC(N)C(=O)O,YGPSJZOEDVAXAB-UHFFFAOYSA-N,L-KYNURENINE
726,CCS(=O)(=O)c1ccc2oc(-c3ccc4ccccc4c3)nc2c1,KSGCNXAZROJSNW-UHFFFAOYSA-N,EZUTROMID
727,COc1ccc2c3c1OC1C(=O)CC[C@@]4(O)[C@@H](C2)N(C)C...,BRUQQQPBMZOVGD-FSFXSCMFSA-N,OXYCODONE
